In [1]:
## Features
initializer = 'Xavier'

In [2]:
import math, random
import gym
import numpy as np
import sys
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd 
import torch.nn.functional as F

In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
sys.path.append('../')
from common.wrappers import make_atari, wrap_deepmind, wrap_pytorch
from common.save_file import *

<h1>Atari Environment</h1>

In [6]:
env_id = "DemonAttack-v0"
env    = make_atari(env_id)
env    = wrap_deepmind(env)
env    = wrap_pytorch(env)

<h3>Use Cuda</h3>

In [7]:
USE_CUDA = torch.cuda.is_available()
Variable = lambda *args, **kwargs: autograd.Variable(*args, **kwargs).cuda() if USE_CUDA else autograd.Variable(*args, **kwargs)

<h2>Replay Buffer</h2>

In [8]:
from collections import deque

class ReplayBuffer(object):
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        state      = np.expand_dims(state, 0)
        next_state = np.expand_dims(next_state, 0)
            
        self.buffer.append((state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        state, action, reward, next_state, done = zip(*random.sample(self.buffer, batch_size))
        return np.concatenate(state), action, reward, np.concatenate(next_state), done
    
    def __len__(self):
        return len(self.buffer)

<h2>Computing Temporal Difference Loss</h2>

In [9]:
def compute_td_loss(batch_size):
    state, action, reward, next_state, done = replay_buffer.sample(batch_size)

    state      = Variable(torch.FloatTensor(np.float32(state)))
    next_state = Variable(torch.FloatTensor(np.float32(next_state)), volatile=True)
    action     = Variable(torch.LongTensor(action))
    reward     = Variable(torch.FloatTensor(reward))
    done       = Variable(torch.FloatTensor(done))

    q_values      = model(state)
    next_q_values = model(next_state)

    q_value          = q_values.gather(1, action.unsqueeze(1)).squeeze(1)
    next_q_value     = next_q_values.max(1)[0]
    expected_q_value = reward + gamma * next_q_value * (1 - done)
    
    loss = (q_value - Variable(expected_q_value.data)).pow(2).mean()
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss

In [10]:
def plot(frame_idx, rewards, losses):
    clear_output(True)
    plt.figure(figsize=(20,5))
    plt.subplot(131)
    plt.title('frame %s. reward: %s' % (frame_idx, np.mean(rewards[-10:])))
    plt.plot(rewards)
    plt.subplot(132)
    plt.title('loss')
    plt.plot(losses)
    plt.show()

<p><hr></p>

In [11]:
class CnnDQN(nn.Module):
    def __init__(self, input_shape, num_actions):
        super(CnnDQN, self).__init__()
        
        self.input_shape = input_shape
        self.num_actions = num_actions
        
        self.features = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )
        
        self.fc = nn.Sequential(
            nn.Linear(self.feature_size(), 512),
            nn.ReLU(),
            nn.Linear(512, self.num_actions)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        
        return x
    
    def feature_size(self):
        return self.features(autograd.Variable(torch.zeros(1, *self.input_shape))).view(1, -1).size(1)
    
    def act(self, state, epsilon):
        if random.random() > epsilon:
            state   = Variable(torch.FloatTensor(np.float32(state)).unsqueeze(0), volatile=True)
            q_value = self.forward(state)
            action  = q_value.max(1)[1].data[0]
        else:
            action = random.randrange(env.action_space.n)
        return action
    
    def predict(self, state):
        state = Variable(torch.FloatTensor(np.float32(state)).unsqueeze(0), volatile=True)
        q_value = self.forward(state)
        return q_value
    
    def weights_init(self, m):
        classname = m.__class__.__name__
        if classname.find('Conv') != -1 and classname.find('Layer') == -1:
            nn.init.xavier_normal(m.weight)
        elif classname.find('Linear') != -1:
            #nn.init.constant(m.weight,0)
            nn.init.xavier_normal(m.weight)
            nn.init.constant(m.bias, 0.1)
        elif classname.find('BatchNorm') != -1:
            m.weight.data.normal_(1.0, 0.02)
            m.bias.data.fill_(0)

In [12]:
model = CnnDQN(env.observation_space.shape, env.action_space.n)
if initializer == 'Xavier':
    model.apply(model.weights_init)
    nn.init.constant(model.fc[2].weight.data,0)
    nn.init.constant(model.fc[2].bias.data,0)
    print('Initialized with Xavier initializer')

if USE_CUDA:
    model = model.cuda()
    print('Using CUDA!')
    
optimizer = optim.Adam(model.parameters(), lr=0.00001)

replay_initial = 200
replay_buffer = ReplayBuffer(300000)

Initialized with Xavier initializer
Using CUDA!


In [13]:
def nn_entropy(p):
    p_softmax = nn.Softmax()(p).data.cpu().numpy()[0]
    return -np.sum(p_softmax*np.log(p_softmax))

In [14]:
tau = 10
entropy_threshold = np.log(env.action_space.n)*0.5
qe_max = np.log2(env.action_space.n)
print('entropy maximum: %.3f' %(qe_max))

entropy maximum: 2.585


In [ ]:
num_frames = 3000000
batch_size = 32
gamma      = 0.99
episode = 0

losses = []
all_rewards = []
episode_reward = 0

state_traj = []
q_value_traj = []
q_entropy_traj = []

episode_state = []
episode_q_val = []
episode_q_entropy = []


state = env.reset()
for frame_idx in range(1, num_frames + 1):
    q_values = model.predict(state)#.data.cpu().numpy()[0]
    q_entropy = nn_entropy(q_values)
    
    
    if frame_idx % 100 == 0:
        QQ= q_values.data.cpu().numpy()[0]
        print('\rframe: %5d, q_entropy: %.3f, (%.2f%% of max)  q_values: %.2f, %.2f, %.2f, %.2f, %.2f, %.2f'
        %(frame_idx, q_entropy, (q_entropy/qe_max)*100, QQ[0], QQ[1], QQ[2], QQ[3], QQ[4], QQ[5]),end='')

    if q_entropy > entropy_threshold:
        action = env.action_space.sample()
    else:
        action = model.act(state=state, epsilon=0)
    
    
    if episode % 20 == 0:
        episode_state.append(state)
        episode_q_val.append(q_values)
        episode_q_entropy.append(q_entropy)
    
    next_state, reward, done, _ = env.step(action)
    replay_buffer.push(state, action, reward, next_state, done)
    
    state = next_state
    episode_reward += reward
    
    if done:
        state = env.reset()
        all_rewards.append(episode_reward)
        episode_reward = 0
        if episode % 20 == 0:
            state_traj.append(episode_state)
            q_value_traj.append(episode_q_val)
            q_entropy_traj.append(episode_q_entropy)
        
        episode += 1
        episode_state = []
        episode_q_val = []
        episode_q_entropy = []
        
    if len(replay_buffer) > replay_initial:
        loss = compute_td_loss(batch_size)
        losses.append(loss.data[0])
        
    if frame_idx % 10000 == 0:
        plot(frame_idx, all_rewards, losses)
        
    #print('frame_idx: ', frame_idx, end='')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


frame:  8800, q_entropy: 0.664, (25.68% of max)  q_values: 231.30, 218.88, 243.85, 242.58, 240.84, 233.37

In [100]:
A=model.predict(state).data.cpu().numpy()[0]
A=np.array([0.1,0.1,0.5,0.1])

In [101]:
np.exp(A)/np.sum(np.exp(A))

array([0.22262668, 0.22262668, 0.33211997, 0.22262668])

In [109]:
entropy(model.predict(state))x

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


0.82629156

In [105]:
ls=nn.LogSoftmax()

a=Variable(torch.Tensor(A))
sm=nn.Softmax()
nn.Softmax()(a)
#sm(a).data.cpu().numpy()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """


Variable containing:
 0.2226
 0.2226
 0.3321
 0.2226
[torch.cuda.FloatTensor of size 4 (GPU 0)]

In [82]:
entropy(A, 10)

1.7890374958515167

In [ ]:
from common.save_file import *

model_dir = "model"
var_dir = "var"
name = "egreedy_" + env_id

save_model(model, model_dir, name)

var_dict = {
            "all_rewards": all_rewards,
            "losses": losses,
            "state_traj": state_traj,
            "q_value_traj": q_value_traj,
            "q_entropy": q_entropy_traj
           }

save_variable(name, var_dir, var_dict)
